# OpenAI Gym (Value iteration algorithm)
https://www.kaggle.com/charel/learn-by-example-reinforcement-learning-with-gym

## Setup

In [1]:
import gym
import numpy as np

## Look at the Taxi-v2 env

In [2]:
env = gym.make('Taxi-v2')
env.env.s = 26
env.render()
env.close()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



__Actions__:
- 0: move south
- 1: move north
- 2: move east 
- 3: move west 
- 4: pick up passenger
- 5: drop off passenger

__Rendering__:
- blue: passenger
- magenta: destination
- yellow: empty taxi
- green: full taxi
- |: wall
- other letters: locations

## Value iteration algorithm
* The value iteration algorithm is centered around the game states;
* It has to know all environment states/transitions upfront (__model-based__ method);
* The core idea of the algorithm is to calculate the value (expected long term maximum results) of each state.

### Bellman Equation

$V^{\ast}(s) = \max \limits_{a} \{ R(s, a) + \gamma \sum\limits_{s'} {P(s'|s, a) V^{\ast}(s')} \}$  

- R(s, a): Reward of action a in state s.
- P(s'|s, a): Probability of going to state s' given action a in state s. The Taxi game actions are deterministic =>  P(...) = 1 always.
- $\gamma$: Discount factor between 0 and 1. Higher $\gamma$ means a higher focus on long term rewards.

In [3]:
env = gym.make('Taxi-v2')

In [4]:
NUM_ACTIONS = env.action_space.n
NUM_STATES = env.observation_space.n
print(f'NUM_ACTIONS: {NUM_ACTIONS}')
print(f'NUM_STATES: {NUM_STATES}')

NUM_ACTIONS: 6
NUM_STATES: 500


In [5]:
v = np.zeros(NUM_STATES)
pi = np.zeros(NUM_STATES, dtype=int)
gamma = 0.9
significant_improvement = 0.01

In [6]:
def best_action(env, s):
    best_a = None
    best_value = float('-inf')
    
    for a in range(NUM_ACTIONS):
        env.env.s = s
        s_next, reward, _, _ = env.step(a)
        value = reward + gamma * v[s_next]
        if best_value < value:
            best_value = value
            best_a = a
    
    env.env.s = s
    return best_a

def value_iteration(env):
    iteration = 0
    while True:
        max_change = 0
        env.reset()
        
        for s in range(NUM_STATES):
            v_old = v[s]
            
            # choose best_action for state s
            best_a = best_action(env, s)
            
            # step with best_action
            env.env.s = s
            s_next, reward, _, _ = env.step(best_a)
            v[s] = reward + gamma * v[s_next]
            
            # save best_a for state s
            pi[s] = best_a
            
            # update max_change
            max_change = max(max_change, abs(v_old - v[s]))
            
        iteration += 1
        if max_change < significant_improvement:
            print(f'{iteration} iterations done')
            break
            
def solve_game(env, pi, s=404):
    total_reward = 0
    done = False
    
    env.reset()
    env.env.s = s
    env.render()
    
    while not done:
        a = pi[s]
        s, r, done, _ = env.step(a)
        env.render()
        print(f'reward: {r}')
        total_reward += r
        
    print(f'total reward: {total_reward}')

In [7]:
value_iteration(env)

41 iterations done


In [8]:
solve_game(env, pi)

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
reward: -1
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
reward: -1
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
reward: -1
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
reward: -1
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
reward: -1
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
reward: -1
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
reward: -1
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
reward: -1
+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+------